## Imorts and Config

In [ ]:
import gc
from pathlib import Path

import albumentations as A
import numpy as np
import pandas as pd
import timm
from albumentations.pytorch import ToTensorV2
from fastai.vision.all import *
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

In [ ]:
TRAIN_CSV = Path('train.csv')
R_TRAIN_DATASET = Path('./data/train_256/')
R_TEST_DATASET = Path('./data/test_256/')
BEST_SUBMISSION = Path('./submissoins/subm_best6.csv')
SAVE_MODELS_DIR = Path('./models/vitbasefold_vhaugs')
USE_TEST_IN_TRAIN = True

In [ ]:
SEED = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

## Data Preparation

In [ ]:
df = pd.read_csv(TRAIN_CSV, delimiter="\t")
df.image_name = df.image_name.apply(lambda x: x.split(".")[0] + ".jpg")
df["path"] = df["image_name"].apply(lambda x: R_TRAIN_DATASET / x)
df = df.sample(frac=1, random_state=SEED)

if USE_TEST_IN_TRAIN:
    df_test = pd.read_csv(BEST_SUBMISSION, delimiter="\t")
    df_test.image_name = df_test.image_name.apply(lambda x: x.split(".")[0] + ".jpg")
    df_test["path"] = df_test["image_name"].apply(lambda x: R_TEST_DATASET / x)
    df = pd.concat([df, df_test]).reset_index(drop=True)


In [ ]:
class AlbumentationsTransform(DisplayedTransform):
    split_idx, order = 0, 2

    def __init__(self, train_aug):
        store_attr()

    def encodes(self, img: PILImage):
        aug_img = self.train_aug(image=np.array(img))["image"]
        return PILImage.create(aug_img)


def A_augs():
    return A.Compose(
        [
            A.ColorJitter(hue=0.1, p=0.5),
            A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=0.5),
            A.GaussNoise(var_limit=(50.0, 150.0), p=0.6),
            A.CoarseDropout(p=0.5),
            A.PixelDropout(0.005, p=0.5),
            A.Downscale(scale_min=0.75, scale_max=0.9, interpolation=4, p=0.3),
            A.ImageCompression(quality_lower=45, quality_upper=85, p=0.3),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.3),
            A.CLAHE(clip_limit=(1, 6), p=0.3),
            A.Sharpen(alpha=(0.1, 0.5), lightness=(0.5, 1.0), p=0.3),
            A.RandomGridShuffle(grid=(3, 3), p=0.1),
        ]
    )


## Train k Models

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

for i, (train_index, val_index) in enumerate(
    skf.split(np.zeros(len(df)), df["label_id"])
):
    print(f"\nFold {i}")

    df["isVal"] = False
    df.loc[val_index, "isVal"] = True

    datablock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_x=ColReader("path"),
        get_y=ColReader("label_id"),
        splitter=ColSplitter("isVal"),
        item_tfms=AlbumentationsTransform(A_augs()),
        batch_tfms=aug_transforms(
            mult=2,
            min_scale=1,
            size=224,
        ),
    )
    dls = datablock.dataloaders(df, bs=32)

    learn = vision_learner(
        dls,
        "vit_base_patch16_224_in21k",
        opt_func=QHAdam,
        metrics=[F1Score(average="macro")],
    )

    filename = "model"
    learn.fine_tune(
        10,
        1e-3,
        freeze_epochs=1,
        cbs=[
            SaveModelCallback(monitor="f1_score", fname=filename, at_end=True),
        ],
    )
    learn.export(SAVE_MODELS_DIR / f"model_{i}.pkl")

    del learn, dls
    gc.collect()
    torch.cuda.empty_cache()
